In [1]:
import numpy as np
import pandas as pd
import string
from collections import defaultdict
import math
from my_utils_pos import preprocessing, get_word_tag

In [2]:
# reading training file
with open("./data/WSJ_02-21.pos") as f:
    training_lines = f.readlines()

training_lines[0:5]

['In\tIN\n', 'an\tDT\n', 'Oct.\tNNP\n', '19\tCD\n', 'review\tNN\n']

In [3]:
# preparing dictionary from the training set.

# words = [line.split("\t")[0] for line in training_lines]

# freq = defaultdict(int)
# for word in words:
#     freq[word] += 1

# training_vocab_l = [k for k,v in freq.items() if (v>1)]

# # training_vocab_l.append('')
# # training_vocab_l.append("--unk--")

# training_vocab_l.sort()

with open("./data/hmm_vocab.txt", 'r') as f:
    training_vocab_l = f.read().split('\n')

print("A few items of the vocabulary list")
print(training_vocab_l[0:50])
print()
print("A few items at the end of the vocabulary list")
print(training_vocab_l[-50:])

A few items of the vocabulary list
['!', '#', '$', '%', '&', "'", "''", "'40s", "'60s", "'70s", "'80s", "'86", "'90s", "'N", "'S", "'d", "'em", "'ll", "'m", "'n'", "'re", "'s", "'til", "'ve", '(', ')', ',', '-', '--', '--n--', '--unk--', '--unk_adj--', '--unk_adv--', '--unk_digit--', '--unk_noun--', '--unk_punct--', '--unk_upper--', '--unk_verb--', '.', '...', '0.01', '0.0108', '0.02', '0.03', '0.05', '0.1', '0.10', '0.12', '0.13', '0.15']

A few items at the end of the vocabulary list
['yards', 'yardstick', 'year', 'year-ago', 'year-before', 'year-earlier', 'year-end', 'year-on-year', 'year-round', 'year-to-date', 'year-to-year', 'yearlong', 'yearly', 'years', 'yeast', 'yelled', 'yelling', 'yellow', 'yen', 'yes', 'yesterday', 'yet', 'yield', 'yielded', 'yielding', 'yields', 'you', 'young', 'younger', 'youngest', 'youngsters', 'your', 'yourself', 'youth', 'youthful', 'yuppie', 'yuppies', 'zero', 'zero-coupon', 'zeroing', 'zeros', 'zinc', 'zip', 'zombie', 'zone', 'zones', 'zoning', '{',

In [4]:
# vocab: dictionary that has index for corresponding words
vocab = {}

for i, word in enumerate(sorted(training_vocab_l)):
    vocab[word] = i

print("Vocabulary dictionary, Key is word and value is a unique integer")
cnt=0
for k,v in vocab.items():
    print(f"{k} : {v}")
    cnt+=1
    if cnt>20:
        break

Vocabulary dictionary, Key is word and value is a unique integer
 : 0
! : 1
# : 2
$ : 3
% : 4
& : 5
' : 6
'' : 7
'40s : 8
'60s : 9
'70s : 10
'80s : 11
'86 : 12
'90s : 13
'N : 14
'S : 15
'd : 16
'em : 17
'll : 18
'm : 19
'n' : 20


In [5]:
_ , prep = preprocessing(vocab, "./data/test.words")

print('The length of the preprocessed test corpus: ', len(prep))
print('This is a sample of the test_corpus: ')
print(prep[0:10])

The length of the preprocessed test corpus:  34199
This is a sample of the test_corpus: 
['The', 'economy', "'s", 'temperature', 'will', 'be', 'taken', 'from', 'several', '--unk--']


## Creating Emission, Transition and Tag count dictionaries

### Transition Counts
- This dictionary will have (prev_tag, tag) as key and the correspnidng tag pair count as value.

### Emission Counts
- This dictionary will have (tag, word) as key and corresponding pair count as value.

### Tag Counts
- This has tag as key and its count as value

In [6]:
def create_dictionaries(trainig_corpus, vocab, verbose=True):
    emission_counts = defaultdict(int)
    transition_counts = defaultdict(int)
    tag_counts = defaultdict(int)

    i = 0

    prev_tag = "--s--"

    for word_tag in trainig_corpus:

        i+=1

        if i%5000==0 and verbose:
            print(f"Word Count = {i}")

        word, tag = get_word_tag(word_tag, vocab)

        emission_counts[(tag, word)] += 1
        transition_counts[(prev_tag, tag)] += 1
        tag_counts[tag] += 1

        prev_tag = tag
    
    return emission_counts, transition_counts, tag_counts

In [7]:
emission_counts, transition_counts, tag_counts = create_dictionaries(training_lines, vocab=vocab)

Word Count = 5000
Word Count = 10000
Word Count = 15000
Word Count = 20000
Word Count = 25000
Word Count = 30000
Word Count = 35000
Word Count = 40000
Word Count = 45000
Word Count = 50000
Word Count = 55000
Word Count = 60000
Word Count = 65000
Word Count = 70000
Word Count = 75000
Word Count = 80000
Word Count = 85000
Word Count = 90000
Word Count = 95000
Word Count = 100000
Word Count = 105000
Word Count = 110000
Word Count = 115000
Word Count = 120000
Word Count = 125000
Word Count = 130000
Word Count = 135000
Word Count = 140000
Word Count = 145000
Word Count = 150000
Word Count = 155000
Word Count = 160000
Word Count = 165000
Word Count = 170000
Word Count = 175000
Word Count = 180000
Word Count = 185000
Word Count = 190000
Word Count = 195000
Word Count = 200000
Word Count = 205000
Word Count = 210000
Word Count = 215000
Word Count = 220000
Word Count = 225000
Word Count = 230000
Word Count = 235000
Word Count = 240000
Word Count = 245000
Word Count = 250000
Word Count = 255000


In [8]:
states = sorted(tag_counts.keys())
print(f"Number of POS tags (number of 'states'): {len(states)}")
print("View these POS tags (states)")
print(states)

Number of POS tags (number of 'states'): 46
View these POS tags (states)
['#', '$', "''", '(', ')', ',', '--s--', '.', ':', 'CC', 'CD', 'DT', 'EX', 'FW', 'IN', 'JJ', 'JJR', 'JJS', 'LS', 'MD', 'NN', 'NNP', 'NNPS', 'NNS', 'PDT', 'POS', 'PRP', 'PRP$', 'RB', 'RBR', 'RBS', 'RP', 'SYM', 'TO', 'UH', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ', 'WDT', 'WP', 'WP$', 'WRB', '``']


In [9]:
print("transition examples: ")
for ex in list(transition_counts.items())[:3]:
    print(ex)
print()

print("emission examples: ")
for ex in list(emission_counts.items())[200:203]:
    print (ex)
print()

print("ambiguous word example: ")
for tup,cnt in emission_counts.items():
    if tup[1] == 'back': print (tup, cnt) 

transition examples: 
(('--s--', 'IN'), 5050)
(('IN', 'DT'), 32364)
(('DT', 'NNP'), 9044)

emission examples: 
(('DT', 'any'), 721)
(('NN', 'decrease'), 7)
(('NN', 'insider-trading'), 5)

ambiguous word example: 
('RB', 'back') 304
('VB', 'back') 20
('RP', 'back') 84
('JJ', 'back') 25
('NN', 'back') 29
('VBP', 'back') 4


# Hidden Markov Models

Creating Transition and Emission matrices

In [10]:
def create_transition_matrix(alpha, tag_counts, transition_counts):

    all_tags = sorted(tag_counts.keys())

    num_tags = len(all_tags)

    A = np.zeros((num_tags, num_tags))

    trans_keys = set(transition_counts.keys())

    for i in range(num_tags):
        for j in range(num_tags):
            count = 0

            key = (all_tags[i], all_tags[j])

            if key in trans_keys:
                count = transition_counts[key]
            
            count_prev_tag = tag_counts[all_tags[i]]

            A[i,j] = (count+alpha)/(count_prev_tag+num_tags*alpha)
    
    return A

In [11]:
alpha = 0.001
A = create_transition_matrix(alpha, tag_counts, transition_counts)

print(f"A at row 0, col 0: {A[0,0]:.9f}")
print(f"A at row 3, col 1: {A[3,1]:.4f}")

print("View a subset of transition matrix A")
A_sub = pd.DataFrame(A[30:35,30:35], index=states[30:35], columns = states[30:35] )
print(A_sub)

A at row 0, col 0: 0.000007040
A at row 3, col 1: 0.1691
View a subset of transition matrix A
              RBS            RP           SYM        TO            UH
RBS  2.217069e-06  2.217069e-06  2.217069e-06  0.008870  2.217069e-06
RP   3.756509e-07  7.516775e-04  3.756509e-07  0.051089  3.756509e-07
SYM  1.722772e-05  1.722772e-05  1.722772e-05  0.000017  1.722772e-05
TO   4.477336e-05  4.472863e-08  4.472863e-08  0.000090  4.477336e-05
UH   1.030439e-05  1.030439e-05  1.030439e-05  0.061837  3.092348e-02


In [12]:
def create_emission_matrix(alpha, tag_counts, emission_counts, vocab: list): # vocab is list

    num_tags = len(tag_counts)
    num_words = len(vocab)

    all_tags = sorted(tag_counts.keys())

    B = np.zeros((num_tags, num_words))

    emis_keys = set(emission_counts.keys())

    for i in range(num_tags):
        for j in range(num_words):
            count = 0

            key = (all_tags[i], vocab[j])

            if key in emis_keys:
                count = emission_counts[key]
            
            count_tag = tag_counts[all_tags[i]]

            B[i,j] = (count+alpha)/(count_tag+alpha*num_words)
    
    return B

In [13]:
 #creating your emission probability matrix. this takes a few minutes to run. 
alpha = 0.001
B = create_emission_matrix(alpha, tag_counts, emission_counts, list(vocab))

print(f"View Matrix position at row 0, column 0: {B[0,0]:.9f}")
print(f"View Matrix position at row 3, column 1: {B[3,1]:.9f}")

# Try viewing emissions for a few words in a sample dataframe
cidx  = ['725','adroitly','engineers', 'promoted', 'synergy']

# Get the integer ID for each word
cols = [vocab[a] for a in cidx]

# Choose POS tags to show in a sample dataframe
rvals =['CD','NN','NNS', 'VB','RB','RP']

# For each POS tag, get the row number from the 'states' list
rows = [states.index(a) for a in rvals]

# Get the emissions for the sample of words, and the sample of POS tags
B_sub = pd.DataFrame(B[np.ix_(rows,cols)], index=rvals, columns = cidx )
print(B_sub)

View Matrix position at row 0, column 0: 0.000006032
View Matrix position at row 3, column 1: 0.000000720
              725      adroitly     engineers      promoted       synergy
CD   8.201296e-05  2.732854e-08  2.732854e-08  2.732854e-08  2.732854e-08
NN   7.521128e-09  7.521128e-09  7.521128e-09  7.521128e-09  2.257091e-05
NNS  1.670013e-08  1.670013e-08  4.676203e-04  1.670013e-08  1.670013e-08
VB   3.779036e-08  3.779036e-08  3.779036e-08  3.779036e-08  3.779036e-08
RB   3.226454e-08  6.456135e-05  3.226454e-08  3.226454e-08  3.226454e-08
RP   3.723317e-07  3.723317e-07  3.723317e-07  3.723317e-07  3.723317e-07


## Vitirbi Algorithm

In [14]:
def initialize(states, tag_counts, A, B, corpus, vocab):
    num_tags = len(tag_counts.keys())

    best_probs = np.zeros((num_tags, len(corpus)))

    best_paths = np.zeros((num_tags, len(corpus)), dtype=int)

    s_idx = states.index("--s--")

    for i in range(num_tags):

        if A[s_idx, i] == 0:
            best_probs[i,0] = float('-inf')
        else:
            best_probs[i,0] = math.log(A[s_idx, i])+math.log(B[i, vocab[corpus[0]]])
    
    return best_probs, best_paths

In [15]:
best_probs, best_paths = initialize(states, tag_counts, A, B, prep, vocab)

In [16]:
print(f"best_probs[0,0]: {best_probs[0,0]:.4f}")
print(f"best_paths[2,3]: {best_paths[2,3]:.4f}")

best_probs[0,0]: -22.6098
best_paths[2,3]: 0.0000


In [17]:
def forward(A, B, best_probs, best_paths, vocab, corpus):

    num_tags = best_probs.shape[0]

    for i in range(1, len(corpus)):

        for j in range(num_tags):

            best_prob_i = float('-inf')
            best_path_i = None

            for k in range(num_tags):
                prob = best_probs[k, i-1] + math.log(A[k, j]) + math.log(B[j, vocab[corpus[i]]])

                if prob > best_prob_i:
                    best_prob_i = prob
                    best_path_i = k
            
            best_probs[j, i] = best_prob_i
            best_paths[j, i] = best_path_i
    
    return best_probs, best_paths

In [18]:
best_probs, best_paths = forward(A, B, best_probs, best_paths, vocab, prep)

In [19]:
print(f"best_probs[0,1]: {best_probs[0,1]:.4f}") 
print(f"best_probs[0,4]: {best_probs[0,4]:.4f}") 

best_probs[0,1]: -24.7822
best_probs[0,4]: -49.5601


In [20]:
best_paths.shape

(46, 34199)

In [21]:
def backward(best_probs, best_paths, corpus, states):
    
    m = best_paths.shape[1]

    z = [None] * m

    num_tags = best_probs.shape[0]

    best_prob_for_last_word = float('-inf')

    pred = [None] * m

    for k in range(num_tags):
        if best_probs[k, -1] > best_prob_for_last_word:
            best_prob_for_last_word = best_probs[k, -1]
            z[m-1] = k
    
    pred[m-1] = states[k]

    for i in range(len(corpus)-1, -1, -1):
        pos_tag_for_word_i = best_paths[np.argmax(best_probs[:,i]), i]

        z[i-1] = best_paths[pos_tag_for_word_i, i]

        pred[i-1] = states[pos_tag_for_word_i]

    
    return pred